In [1]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds

from tensorflow import data as tfdata
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import optimizers
from tensorflow import losses
from tensorflow.keras import initializers as init

from tensorflow.keras.utils import plot_model

## 图执行模式 : tf.function
tensorflow2.0 默认使用即时执行模式(Eager Execution) 这种模式能够带来灵活及易于调试的特性    
但在特定的场合，如追求高性能或者部署模型时，依然希望使用tensorflow1.x中的图执行模式(Graph Execution), 将模型转化为高效的tensorflow图模型  

tensorflow 2 提供了tf.function模块，结合AutoGraph机制，只需奥加入@tf.function修饰符，就可以将模型以图执行方式执行

** 并不是任何函数都可以被@tf.function修饰 **  
@tf.function 使用静态编译将函数内的代码转化成计算图   
因此对函数内可使用的语句有一定的限制，且需要函数内的操作本身能够被构建为计算图。 
建议在函数内值使用tf原生操作，不要使用过于复杂的python语句  
函数参数只包括tensor或者numpy数组，并最好能够按照计算图的思想取构建函数  



In [3]:
class MNistLoader():
    def __init__(self):
        # 定义mnist对象
        mnist = tf.keras.datasets.mnist
        # 读取mnist数据
        (self.train_data, self.train_label), (self.test_data, self.test_label) = mnist.load_data()
        # mnist的数据为 28*28的0~255的值, 需要将图像转化为[张数， 宽度， 高度， 通道]， 并归一化
        self.train_data = np.expand_dims(self.train_data.astype(np.float32)/255.0, axis=-1)
        self.test_data = np.expand_dims(self.test_data.astype(np.float32)/255.0, axis=-1)

        self.train_label = self.train_label.astype(np.float32)
        self.test_label = self.test_label.astype(np.float32)

        # 统计相关信息
        self.num_train_data, self.num_test_data = self.train_data.shape[0], self.test_data.shape[0]
    
    def get_batch(self, batch_size):
        # 从train_data中随机抽出batch_size个数据
        index = np.random.randint(0, self.num_train_data, batch_size)
        # np数组可以接受list索引，返回list索引对应的值, 原生数据不支持
        return self.train_data[index, :], self.train_label[index]


class CNN(tf.keras.Model):
    def __init__(self):
        # 继承父类的属性和方法
        super().__init__()
        # 定义网络层
        self.conv1 = layers.Conv2D(
            # 卷积层神经元（卷积核）数目, 默认使用kernel_initializer默认使用glorot_uniform进行初始化，简单来讲就是有32个不一样的卷积核，可以指定kernel_initializer
            filters=32,
            # 感受野大小
            kernel_size=[5, 5],
            # padding策略, same/vaild, same表示图片卷积后大小不变，valid表示卷积后图像减小（不对标远进行填充）
            padding='same',
            # 激活函数
            activation=tf.nn.relu
        )

        self.pool1 = layers.MaxPool2D(
            pool_size=[2,2],
            strides=2
        )

        self.conv2 = layers.Conv2D(
            filters=64,
            kernel_size=[5,5],
            padding="same",
            activation=tf.nn.relu
        )

        self.pool2 = layers.MaxPool2D(pool_size=[2,2], strides=2)
        # TODO, 和Flatten的区别是什么
        self.flatten = layers.Reshape(target_shape=(7*7*64, ))
        self.dense1 = layers.Dense(units=1024, activation=tf.nn.relu)
        self.dense2 = layers.Dense(units=10)
    
    def call(self, inputs):
        x = self.conv1(inputs)
        x = self.pool1(x)
        x = self.conv2(x)
        x = self.pool2(x)
        x = self.flatten(x)
        x = self.dense1(x)
        x = self.dense2(x)
        output = tf.nn.softmax(x)
        return output


In [9]:
import time

# 定义超参
num_batches = 400
batch_size = 50
learning_rate = 0.001

# 实例化对象
data_loader = MNistLoader()
model = CNN()

optimizer = optimizers.Adam(learning_rate=learning_rate)

@tf.function
def train_one_step(X, y):
    with tf.GradientTape() as tape:
        y_pred = model(X)
        loss = losses.sparse_categorical_crossentropy(y_true=y, y_pred=y_pred)
        loss = tf.reduce_mean(loss)
        # 这里使用tf内置的tf.print()方法， tf.function不支持Python原生的print
        # if count%100 == 0:
        #     tf.print("loss:", loss)
    grads = tape.gradient(loss,model.variables)
    optimizer.apply_gradients(grads_and_vars=zip(grads, model.variables))

start_time = time.time()
count = 0
for batch_index in range(num_batches):
    X, y = data_loader.get_batch(batch_size)
    train_one_step(X, y)
    count += 1
end_time = time.time()
print(end_time-start_time)

29.30221199989319


运行400个batch进行测试，加入@tf.function后为29秒，不加为36秒  
一般而言，当模型由较多小操作组成时，@tf.function会带来较大的提升效果  
而当模型的操作数量较少时，但单一操作均很耗时时，@tf.function带来的性能的提升不会很大

### tf.function的内在机制 
当被@tf.function修饰的函数第一次被调用的时候，进行以下操作：  
- 在即时模式关闭的环境下，代码依次运行，（只定义了计算节点，没有进行任何实质运算）
- 使用AutoGraph将python控制流语句转化为tf计算图中的对应节点，如 if-->tf.cond   while/for ---> tf.while
- 基于上面两步，建立函数内代码的计算图表示 
- 运行一次计算图 
- 基于函数的名字和输入的函数参数的类型生成一个哈希值，并将建立的计算图缓存到一个hash表中
- 在被@tf.function修饰后的函数再次被调用时，根据***函数名***核输入的***函数参数类型***计算函数hash值，检查表中是否由对应计算图的缓存。如果有则直接使用，否则重建

### AutoGraph: 将Python控制流转化为tensorflow的计算图
使用tf.AutoGraph 模块的底层api tf.autograph.to_code将函数转化为计算图

In [10]:
@tf.function
def square_if_positive(x):
    if x>0:
        x = x*x
    else:
        x = 0
    return x

a = tf.constant(1)
b = tf.constant(-1)

print(square_if_positive(a), square_if_positive(b))
print(tf.autograph.to_code(square_if_positive.python_function))


tf.Tensor(1, shape=(), dtype=int32) tf.Tensor(0, shape=(), dtype=int32)
def tf__square_if_positive(x):
  do_return = False
  retval_ = ag__.UndefinedReturnValue()
  with ag__.FunctionScope('square_if_positive', 'square_if_positive_scope', ag__.ConversionOptions(recursive=True, user_requested=True, optional_features=(), internal_convert_user_code=True)) as square_if_positive_scope:

    def get_state():
      return ()

    def set_state(_):
      pass

    def if_true():
      x_1, = x,
      x_1 = x_1 * x_1
      return x_1

    def if_false():
      x = 0
      return x
    cond = x > 0
    x = ag__.if_stmt(cond, if_true, if_false, get_state, set_state, ('x',), ())
    do_return = True
    retval_ = square_if_positive_scope.mark_return_value(x)
  do_return,
  return ag__.retval(retval_)



*** AutoGraph起到的时类似编译器的功能，使用户能更加自然的使用python的控制流构建有条件循环， 而无需使用tensorflow的API进行构建***

### 传统的tf.Session
tensorflow 2.x 提供了tf.compat.v1模块用以支持tensorflow 1.x版本的api  
keras的模型是同时可以兼容Eager和Graph模式的。  
在图模式下，model(input_tensor)只需运行依次以完成图的建立操作

In [ ]:
optimizer = tf.compat.v1.train.AdamOptimizer(learning_rate=learning_rate)
    num_batches = int(data_loader.num_train_data // batch_size * num_epochs)
    # 建立计算图
    X_placeholder = tf.compat.v1.placeholder(name='X', shape=[None, 28, 28, 1], dtype=tf.float32)
    y_placeholder = tf.compat.v1.placeholder(name='y', shape=[None], dtype=tf.int32)
    y_pred = model(X_placeholder)
    loss = tf.keras.losses.sparse_categorical_crossentropy(y_true=y_placeholder, y_pred=y_pred)
    loss = tf.reduce_mean(loss)
    train_op = optimizer.minimize(loss)
    sparse_categorical_accuracy = tf.keras.metrics.SparseCategoricalAccuracy()
    # 建立Session
    with tf.compat.v1.Session() as sess:
        sess.run(tf.compat.v1.global_variables_initializer())
        for batch_index in range(num_batches):
            X, y = data_loader.get_batch(batch_size)
            # 使用Session.run()将数据送入计算图节点，进行训练以及计算损失函数
            _, loss_value = sess.run([train_op, loss], feed_dict={X_placeholder: X, y_placeholder: y})
            print("batch %d: loss %f" % (batch_index, loss_value))

        num_batches = int(data_loader.num_test_data // batch_size)
        for batch_index in range(num_batches):
            start_index, end_index = batch_index * batch_size, (batch_index + 1) * batch_size
            y_pred = model.predict(data_loader.test_data[start_index: end_index])
            sess.run(sparse_categorical_accuracy.update_state(y_true=data_loader.test_label[start_index: end_index], y_pred=y_pred))
        print("test accuracy: %f" % sess.run(sparse_categorical_accuracy.result()))